# Design guide
This document establishes design guidelines to be implemented during the construction of all elements of the *Graboid* program.

## Program outline
Briefly, *Graboid* is a machine learning sequence classifier that employs unsupervised methods to assign taxonomic labels to DNA sequence records. As such, it's two main functions are:

* Training
* Classification

Additionally, it includes the functions of

* Reference database construction
* Reference database assessment
* Parameter selection

## Classification algorithms
*Graboid* employs two classification algorithms:

* K-NN
* Concept learning

# K-NN
The K-NN algorithm places training and query instances in an $N$-dimensional sample space determined by the set of $N$ selected attributes in the dataset. Each instance's position in the sample space is determined by its values at the selected attributes. The core assumption of the algorithm is:
> Instances belonging to a given class $C$ will exibit simmilar values at the selected attributes and therefore will cluster together in the sample space

Given a query instance $Q$ beloning to an undetermined class $C_q$, the value of $C_q$ can be inferred by placing $Q$ in the sample space and determining the most frecuent class value among the $K$ nearest neighbours of $Q$, with $K$ being an arbitrary number of neighbours.

## Distance calculation
The K-NN algorithm hinges on the quantification of *dissimilitude* or *distance* between pairs of instances. The distance $D$ between two instances $A$ and $B$ of length $n$ is calculated with the equation:
$$D(A,B) = \sum_{i=1}^n d(A_i, B_i)$$

Where $d(A_i, B_i)$ is the difference between values at the $i$-th attribute of $A$ and $B$. For numeric and/or ordinal attributes, calculating the difference between $A_i$ and $B_i$ is a simple matter of arithmetic difference.

The distance calculation between categorical non-ordinal attributes requires the employment of sets of *rules of substitution* that assign values of dissimilitude to all value pairs possible when comparing observations of these attributes.

### Identity
Differentiation by *identity* is the simplest way to measure distance between categorical attributes. The calculation consists simply in assigning a distance/difference score of 1 when two instances have differing values in a given attribute, and a value of 0 if they do not.
$$
d(a,b) = 
\begin{cases}
0 \text{ if } a=b\\
1 \text{ if } a \ne b
\end{cases}
$$

### Substitution models
The problem with using identity as a mesure of differentiation is that it assigns the same distance value to all possible substitutions, which may not be the case in the context of comparing genetic sequences. Special consideration can be given to different pairs of values observed to extract more detailed information about distances between sequences.

One such consideration is the *K2P* model, which aims to take the different likelihoods of the two types of substitutions that are possible between DNA sequences: transitions and transversions. The model assigns a distance score of 1 to substitutions between bases of the same *type* ($purine \rightarrow\ purine$, $pyrimidine\ \rightarrow\ pyrimidine$), and a score of 2 to substitutions between bases of differing types ($purine \leftrightarrow\ pyrmidine$).

$$
d(a,b) = 
\begin{cases}
0 \text{ if } a=b\\
1 \text{ if } a \ne b\ \&\ type(a) = type(b)\\
2 \text{ if } a \ne b\ \&\ type(a) \ne type(b)
\end{cases}
$$

### Distance matrices
Since the distance values between every possible value pair are known *a priori*, they can be stored in a $4 \times 4$ matrix in which each cell correspond to a given combination of bases (*A*, *C*, *G*, *T*). A fifth row and column are added to account for missing or unknown data.

Using the resulting distance matrix $M_{5,5}$, the distance calculation between $A$ and $B$ becomes:
$$D(A,B) = \sum_{i=1}^n M_{[A_i, B_i]}$$

For classification by identity, $M$ takes the form of $M_{id}$:
$$
M_{id} = \left [
\begin{matrix}
0 & 1 & 1 & 1 & .75 \\
1 & 0 & 1 & 1 & .75 \\
1 & 1 & 0 & 1 & .75 \\
1 & 1 & 1 & 0 & .75 \\
.75 & .75 & .75 & .75 & .75
\end{matrix}
\right ]
$$

While for classification by the K2P model, $M$ takes the form of $M_{K2P}$:
$$
M_{K2P} = \left [
\begin{matrix}
0 & 2 & 2 & 1 & 1.25 \\
2 & 0 & 1 & 2 & 1.25 \\
2 & 1 & 0 & 2 & 1.25 \\
1 & 2 & 2 & 0 & 1.25 \\
1.25 & 1.25 & 1.25 & 1.25 & 1.25
\end{matrix}
\right ]
$$

Notice the fifth row and columns of the matrices. These correspond to the *missing* or *unknown* values. When comparing a known value with an unknown, an intermediate distance value is given. This value is taken as the average of possible values, including the coincidence.

## Training
The training stage for a K-NN algorithm consists simply of loading the reference dataset $R$, which is comprised of a set of $r$ instances of $n$ attributes and a corresponding array $R_y$ of length $r$ containing the class value for each instance of $r$. Taxonomic classifications also present a hierarchical organization with multiple ranks. To acomodate for this, $R_y$ becomes a 2D array of $r$ rows and $u=$ columns with $u$ being the number of available taxonomic ranks. Classifications are produced for every rank.

### Sequence collapsing
When selecting a section of a sequence alignment, it is possible that some instances have a complete coincidence along the selected region. That is, they are effectively indistinguishable within the selected section. In this case, multiple instances may occupy the same place in the instance space. This may not only contribute to artificially inflate the presence of a given class in the vicinty of the query instance, but also might generate a classification conflict if instances of different classes coincide in the selected region.

To adress this isue, the reference set $R$ is *collapsed*. That is, the sequences in $R$ are clustered by 100% coincidence in *known* attributes (unknown values in one or both instances compared are not counted as differing values).

$R_y$ is collapsed accordingly. For each resulting cluster, classification conflicts are identified and resolved. A *classification conflict* happens when a given cluster contains representatives of different classes at a given taxonomic rank. When one such case is encountered, the class value for the cluster is set as *unknown* at the rank in which the conflict took place, as well as all the ranks below it.

The collapsed $R$ and $R_y$ are known as $R'$ and $R_y'$. The dimensions of these new matrices are $r' \times n$ and $r' \times u$ respectively.

### Feature selection
The amount of information provided may differ among the available attributes (sites of the alignment). In the worst possible case, an attribute is invariable: it presents a single *known* value for all reference instances. Such an attribute has no use in the classification, as it cannot be used to subdivide the alignment.

Opposite to this, an ideal attribute is one that presents multiple values, each one characteristic to a *taxonomically consistent* (all sequences belonging to the same taxon) subset of the reference set.

In order to determine the utility of any given site an information quantification function based on the *Shannon Entropy index* is implemented. As a reminder, Snannon's Entropy $I$ for a given array $A$ is calculated as:

$$I = -\sum_{i \in \{A\}} p_i\ log_2 p_i$$

The $I$-index value increases with the amount of values present in $A$ and with the uniformity of their distributions. The $I$-index measures the diversity within a given attribute; by itself is it not enough to determine whether a site is useful in the classification problem. The utility of $A$ depends on how its internal diversity is distributed among the representatives of the different classes to be identified; this distribution can be calculated as the the *information gain* $G_t$:

$$G_t = I_{gen} - I_{!t}$$

Where $I_{gen}$ is the value of entropy for the entire alignment. And $I_{!t}$ is the entropy for the alignment minus all the representatives of a given taxon $t$.

The $G_t$ metric indicates how the removal of $t$ affects the information content in $A$. Such effects depend on both the diversity of $t$ and the alignment as a whole:

|#|Case|Expected $G_t$ value|Reason|
|:-|:-:|:-:|:-:|
|1|$A$ contains a single known value|0|Both $I_{gen}$ and $I_{!t}$ are 0|
|2|$t$ contains a single value which is not observed in the rest of the alignment|Highest|The $I$-index value is most resented by the removal of entire categories, $I_{!t}$ would show the greatest reduction in this case|
|3|$t$ contains a single value which is observed in the rest of the alignment and belongs to a *majoritary* class|Negative|The removal of $t$ would contribute to make $A$ more equitative. This would increase the value of $I_{!t}$ with respect to $I_{gen}$ and $G_t$ would present a negative value|
|4|$t$ contains a single value which is observed in the rest of the alignment and belongs to a *minotitary* class|High|The resulting array from removing $t$ would be less equitative than the original. The expected value of $G_t$ would be high, depending on what proportion of the minoritary class was attributable to $t$|
|5|$t$ contains multiple values and at least one of them is not observed in the rest of the alignment|High, Very High|If multiple of the observed values are unique to $t$, the $G_t$ value could be higher than in case 2|
|6|$t$ contains multiple values, none of which are unique to $t$|Low|If the values within $t$ are equitatively distributed, the $G_t$ value would be close to 0, as the relative frequencies would remain mostly unchanged after the removal of $t$|

The feature selection process consists of selecting the $N$ most informative attributes for each taxon present in $R_y$. The set of selected attributes for $t$, $S_t^{(N)}$ is built as:
$$ G_t = [G_t^{(1)}, G_t^{(2)},..., G_t^{(n-1)}, G_t^{(n)}]$$
$$S_t^{(N)} = argsort(G_t)[-N:]$$
with $G_t^{(i)}$ being the information gain for taxon $t$ in the $i$-th site.

The set of attributes selected for the classification is the union of the sites selected for every taxon:
$$S = \bigcup_{i=1}^{|T|}S_i^{(N)}$$

Note that the potential amount of selected sites ranges between $N$ and $N \times |T|$. The fewest possible amount of sites would occur when the same $N$ sites are deemed to be the most informative for every taxon in the reference dataset. The highest amount of sites would be selected when each taxon's $S_t^{(N)}$ set is unique and has no superposition with the rest of the selected site sets.

## Classification
After the training stage is done, the agent is ready to classify unknown sequences. The first step to do this is to build a *query alignment* using the sequence file to be classified. The query alignment is generated by aligning the query sequences against the same guide sequence used to generate the reference alignment. Along with the query alignment, a query id array $Q_{id}$ containing the identifier code of each query sequence is generated. The final query set $Q$ is built by extracting the selected columns $S$ from the query alignment.

For each query sequence $q \in Q$, a distance array $D_q$ containing the distance between $q$ and every reference instance in $R'$ is generated:
$$D_q = [D(q, r_1), D(q, r_2),..., D(q, r_{r'})]$$
$$D(q, r_i) = \sum_{j \in S} M_{[q_j, r_{ij}]}$$

The set of selected neighbours for $q$ is stored as
$$neighbours_q^{(K)} = argsort(D_q)[:K]$$

### Neighbour selection criterion
Due to the categorical nature of sequence data, distances between instances accumulate in discreet steps resulting in the neighbours of $q$ to be arranged in levels of distance or "orbitals" around $q$ in the $n$-dimension sample space. This presents a problem at the moment of selecting the titular $K$ nearest neighbours, as more often than not it is possible that more than $K$ neighbours occupy the first orbital (failing that, less than $K$ neighbours could fill the inner orbitals, but the orbital containing the $K$-th neighbour includes additional neighbours). In order to adhere strictly to the $K$ neighbour criterion it would be necessarily to arbitrarily discard neighbours until only $K$ are considered.

Two alternative criteria are deployed to circunvent this issue:

#### Neighbour criterion
The first criterion consists on selecting orbitals *up until* and *including* the one that contains the $K$-th neighbour. All the neighbours included in the selected orbitals are considered in the classification, regardless if the final number of neigbhours exceeds $K$. This criterion tends to be more conservative than the next one, provided the innermost orbitals are sufficiently populated and the selected value of $K$ is not very high.

In this method, neighbours are selected as those with a distance to $q$ that is *lesser or equal* to $D_q[K]$

#### Orbit criterion
The second criterion consists on selecting the $K$ innermost orbitals, regardless of the number of neighbours included. This method tends to include more neighbours than the previous one unless the inner orbitals are sparsely populated and the selected value of $K$ is not very high.

Here, neighbours are grouped by their distance to $q$ and the $K$ nearest groups are selected.

#### Neighbour weighting
In the original implementation of K-NN, the final classification of $q$, $q_y$ is determined through a *majority vote* of the selected neigbhours: The class with the most observation among the $K$ nearest neigbhours is assigned.
$$T_q = \{R_y'[l \in neighbours_q^{(K)}]\}$$
$$T_q^{count} = count\_uniques(R_y'[l \in neighbours_q^{(K)}])$$
$$q_y = T_q[argmax(T_q^{count})]$$

Where $T_q$ contains the set of taxa present among the selected neighbours and $T_q^{count}$ contains the amount of observations for each taxon in $T_q$

Classes in real datasets often have heterogeneous distributions, with some taxa being limited to a single representative. In cases such as these, the number of included neighbours might be higher than the number of actual representatives of the real taxon. This would result in cases where it is impossible for the classifier to produce a correct result.

This effect can be mitigated by *weighting* the neighbours. That is, ponderating their impact in the final classification according to their proximity to $q$. It stands to reason that reference instances that are closer to $q$ are more likely to belong to the same category of $q$ than instances that are further away. In fact, the original K-NN algorithm is based on this asumption.

In addition to the traditional *majority vote*, classification may be performed using *weighted K-NN*. In this method, each taxon $t$ represented among $T_q$ has a calculated *support* $\phi_t$, which is the sum of the weight of each of its representatives:
$$t_q = [l\ \forall\ l \in\ neighbours_q^{(K)}\ if\ R_y'[l] = t]$$
$$t_q^{distances} = D_q[l]\ \forall\ l \in t_q$$
$$\phi_t = w(t_q^{distances})$$
with $w$ being the weighting function. The taxon presenting the highest $\phi$ value is assigned as $q_t$.

Two weighting functions are implemented:
$$wknn = \sum_{i=1}^{k}\frac{d_k - d_i}{d_k - d_1}$$
$$dwknn = \sum_{i=1}^{k}\frac{d_k - d_i}{d_k - d_1} \times \frac{d_k + d_1}{d_k + d_i}$$

These functions take as input an ordered array of distances $d$ of length $k$, with $d_1$ being the smallest distance and $d_k$ the greatest distance.

The set of supports for all candidate taxa for $q_y$ is stored as:
$$\phi_q = [\phi_1, \phi_2,..., \phi_{|T_q|}]$$

And $q_y$ is defined as:
$$q_y = T_q[argmax(\phi_q)]$$

## Calibration
The amount of taxonomic signal present in a sequence alignment may vary among regions and taxa. Thus the parameters needed to recognize specific clades may be influenced by the region of the alignment being worked with. It is useful to perform an evaluation of the reference dataset to assess the expected classification performance for the represented taxa, as well as the parameter values that yield the best results.

### Grid search
The adjustable parameters in the Graboid implementation of K-NN are (*K*, *N*, *method* and *criterion*). A range of values for each parameter is used to build a grid of shape $params(K) \times params(N) \times params(method) \times params(criterion)$, with each cell corresponding to a unique combination of values. For each cell, a classifciation simulacra is performed to assess classification performance.

### Leave-one-out
The *leave-one-out* method consists of extracting a single instance from the reference dataset and using the remaining sequences to classify it using the remaining instances as reference. This process is repeated for all the instances in the dataset.

### Performance assessment
The classification performance is evaluated using the metrics *accuracy*, *precision*, *recall* and *F score*

# Concept learning
## Prelude
While the K-NN algorithm is a well established classification method it is not without drawbacks, especially for the problem of taxonomic assignment. In the first place, K-NN is apt to work on ordinal, continuous data (most usually, numerical values); while the data presented in DNA sequences is discreet and unordered (namely, the four nitrogenated bases). This results in the need to employ workaround methods to estimate the distances between instances, such as the simplistic *identity* method or the arbitrary *model-based* method. Furthermore, the discreet and restricted nature of the differences between instances results in an orbital-like ordering of the neighbours of a given query, in which multiple reference instances are equidistant to the query despite being located in different places of the sample space. In this case, adhering to the $K$ nearest neighbours classification may not be feasible if the nearest orbital to the query is populated by more than $K$ neighbours.Graboid implements a workaround to this problem at the expense of distorting the value of $K$ selected neighbours.

The second drawback involves a problem specific to the K-NN algorithm: heterogeneous representation of the classes in the dataset. The number of recorded sequences in a reference database varies greatly depending on taxa and taxonomic rank. This is the result of both variable sampling effort (as some taxa tend to be more regularly sequenced than others) and variable degrees of internal sequence diversity between taxa. It is important to note that each instance in a Graboid reference dataset corresponds not to an indiviual organism but to a unique sequence variant of a given molecular marker. Therefore, the number of representatives of a given class are an indicator of its internal diversity rather of its abundance. This is relevant because one of the main criteria for selecting sequences as molecular markers is that they ought to have a low degree of internal diversity in relation to overall diversity, that is, taxa are espected to have few representatives in the sample space. The core notion of "class neighbourhoods" emploed by K-NN may not be applicable if most classes are comprised of a signle representative.

In conclusion, in order to address the weaknesses of K-NN (particularly severe at lower taxonomic ranks), an alternative classification method is needed. This method should be built to estimate simmilitude between instances using categorical data and not be susceptible to heterogeneous class populations.

## Introduction
The classification prolem can be presented as: *Given a query $q$ and a set of classes $C$, to which class $c \in C$ is $q$ most likely to belong to?*

Given the criteria previously established (categorical data and independence of class population sizes in the reference dataset), the chosen approach is the one employed in *Concept learning*.

> A concept $C$ is a category of objects that satisfy a set of specific rules $R_C$.
>
> A given object $o$ is said to be a representative of concept $C$ if it is consistent with all the rules in the set $R_C$.
>
> A rule $r_C$ restricts the values that can be observed in a given attribute for an instance to be consistent with the concept $C$
> 
> An object may belong to multiple concepts.

The learning aspect of concept learning consists of identifying the rules of each concept presented in the training datset.

For the Graboid implementation of concept learning, the different taxa $t \in T$ represented in the reference dataset take the place of concepts, and the sites in the alignment are the attributes from which the rules will be extracted.

Since the reference dataset contains taxonomic information at multiple ranks, concepts are expected to present a *nested* distribution and instances to belong to multiple concepts at the different levels.

### A note on genetic diversity
Genetc diversity at the intra and inter taxonomic levels is distributed heterogeneously across genomes, deriving in the existence of conserved and variable genomic regions. Genetic markers, and therefore the reference dataset, consist of a restricted portion of the genome that may not contain the information required to wholly discriminate between the represented taxa. It is possible that groups of taxa present no distinguishable sequences at the specified genomic region.

## The learning task
Recall that a rule consists of a restriction of the observed values $V_{r_C} = \{V_{r_C}^{(1)}, V_{r_C}^{(2)}, ...\}$ in a given attribute $A_{r_C}$ that can be used to determine whether an instance is consistent with a given concept. A given rule $r_C \in R_C$ may be expressed as:
> Only instances that present values $v \in V_{r_C}$ at the attribute $A_{r_C}$ are consistent with concept $C$

Ideally, every concept in the dataset presents *at least* one attribute that is useful to discriminate its representatives from the remaining instances. However, it is possible that a concept does not present ideal attributes, or that it contains such a degree of internal diversity that some of its representatives do not present ideal attributes. In these cases, the concept may still be identifyable if there exists a combination of attributes that is unique to it.

For a given taxon $t$, sites (attributes) $\{s_1, s_,2, ... \} \in S$ may be typified by their *degree of specificity* with respect to the whole dataset. The characteristics of each possible site type are shown in the following table.

| Type | Distribution | |
|:-:|:-:|:-|
|1|$V_t^s \cap V_{!t}^s = \emptyset$|The values observed in $t$ are *not* observed in any other sequence of the alignment. It is possible for a taxon to present multiple typical values in the same site|
|2|$$V_t^s \cap V_{!t}^s \ne \emptyset$$ $$V_t^s \not\subset V_{!t}^s$$|There exists a *partial* intersection between the values presented by $t$ and the ones encountered in the rest of the alignment|
|3|$$|V^s| > 1$$ $$|V_t^s| = 1$$ $$V_t^s \subset V_{!t}^s$$|Taxon $t$ shows a single known value and it is that is also observed outside of $t$. The site $s$ must still be variable|
|4|$$|V_t^s| > 1$$ $$V_t^s \subseteq V_{!t}^s$$|Taxon $t$ presents multiple values, *all* of which are observed outside of $t$|

$V^s$ indicates the set of values observed in a site $s \in S$, with $V_t^s$ the set of values of $V^s$ that are observed in instances of $t$, and $V_{!t}^s$ the set of $V^s$ values observed in instances not beloning to $t$.

### Rule identification
#### Single rules
For each taxon $t$, rule identification begins by identifying sites that present distinctive values for $t$, that is *type 1/2* sites. These sites allow discrimination of representative sequences of $t$ from the rest of the alignment. For each of these found sites, a *single site* rule is defined (that is, a rule comprised of a single *type 1/2* site).

A single site rule *r* based on a *type 2* site will only allow the discrimination of $t$ representatives that hold the distinctive values at its chosen site, therefore it is called a *single partial* rule. A single rule based on a *type 1* site may allow discrimination of the entire taxon $t$, provided its value is known for every representative; in this case this rule is called a *single whole* rule. If the *type 1* site has unknown values among representatives of $t$, $r$ is deemed to be a *single partial* rule.

#### Composite rules
If a given taxon $t$ contains no *single whole* rules and the union of all its *single partial* rules leaves representatives of $t$ indistinguishable from rest of the alignment, full discrimination may still be achieved if there exists a combination of *type 3* sites that is unique to the undifferentiated representatives of $t$. Such rules are denominated as *composite* rules, and just like *single* rules, there may be *composite whole* and *composite partial* rules that allow to differentiate some or all the representative sequences of $t$ respectively.

The following table summarizes the different rules that may be identified:

|Rule|Sites|Discrimintates|
|:-:|:-:|:-:|
|single whole|One|All|
|single partial|One|Some|
|composite whole|Multiple|All|
|composite partial|Multiple|Some|

The algorithm for constructing a composite rule works as follows:

0) Define *composite_rule* as an empty list
1) Define *sites_3* as the set of all *type 3* sites for $t$
2) Define *out_seqs* as the set of all extraneous sequences (not belonging to $t$) that cannot be distinguished from $t$ using the available *single partial* rules
3) For every site *s3* in *sites_3*, count the sequences within *out_seqs* that share $t$'s value at *s3*
4) Select the site with the fewest shared sequences and add it to *composite_rule*
5) Remove all sequences from *out_seqs* that *don't* share $t$'s value at the selected site
6) Remove selecte site from *sites_3*
7) If there are no sequences remaining in *out_seqs*: End program, produced rule is *composite whole*.
8) If there are no sites remaining or no sequences are removed from *out_seqs*: End program, produced rule is *composite partial*
9)  Go to step 3

This algorithm works by iteratively selecting the *type 3* site that allows to reject the largest number of confused extraneous sequences. The search continues until:

* All extraneous sequences have been rejected, resulting in a *composite whole* rule
* All *type 3* sites are exhausted, resulting in a *composite partial* rule if $t$ hasn't been fully discriminated by this point
* No new extraneous sequences can be rejected after the previous iteration, meaning no new information can be extruded from the remaining *type 3* sites. This results in a *composite partial* rule

For a given taxon, the union of identified rules may allow for the complete discrimination of its representatives from the rest of the alignment, in which case it is designated a *fully solved* taxon. If there are still extraneous sequences that cannot be distinguished, the taxon is labeled as *partially solved* and the non distinguished extraneous sequences are known as *confused*.

Extraneous instances consistent with the rules of $t$ are *type I* error (false positive) with respect to $t$. *Type II* errors should not occur, provided the reference alignment is an accurate sample of the taxon's real genetic diversity. A false negative, meaning a sequence of $t$ not being recognized as such would result from one or more of the identified rules of $t$ being too strict.

### Consistent vs Confirmed
*Single partial* rules built using type 2 sites and *composite* rules will include the number of extraneous sequences which contain a shared value with $t$ at the rule site as potential members of the concept taxon. Sequences that contain this shared value are said to be *consistent* with the concept taxon at the given site. Hypothetically, an extraneous sequence may be consistent with the concept taxon at all rule sites (in this case, the concept taxon has no *single whole* rules), this sequence is *confused* by the concept taxon.

> Extraneous sequences that share a value with a taxon concept in every rule site cannot be distinguished from the concept and are said to be *confused* with it.

It is possible that one or more representatives of the concept taxon have no distinctive values at any of the rule sites. These sequences can at best be said to be *consistent* with the concept taxon, but are functionally identical with a *confused* extraneous sequence. Altenratively, any sequence that contains at least one distinctive value of the concept taxon is *confirmed* to be a representative.

> A representative sequence of a concept taxon that contains at least one distinctive value of the concept is *confirmed*
>
> A concept taxon for which all representative sequences are *confirmed* is said to be *fully solved*
> 
> A concept taxon for which some representatives are not *confirmed* is *partially solved*
>
> A concept taxon for which there are no distinctive values and only has a composite partial rule is *unsolved*

When a concpet taxon is *fully solved*, potential *consistent* sequences can be rejected, as every representative of the taxon is expected to contain at least one dstinctive value.

## Classification
When classifying a query sequence, its values are evaluated against the ruleset of every concept represented in the reference database. Classification results are:

|Result|Description|
|:-|:-|
|Confirmed for $t$|Query sequence contains at least one distinctive value for $t$|
|Compatible with $t$, $t$ not fully solved|Query sequence contains no distinctive values for $t$, but all its values are allowed by the $t$ ruleset. This query is considered a *potential* member of $t$|
|Compatible with $t$, $t$ is fully solved|Query sequence contains no distinctive values for $t$ and all its values are allowed by the $t$ ruleset. All representatives of $t$ are expected to contain at least one distinctive value|
|Not compatible with $t$|Query sequence contains at least one value that is diallowed by the $t$ ruleset and is rejected as a potential member of the concept taxon|

### Hypothesis building
Having determined the content of information for each site, the next step is to implement a way to leverage this knowledge in the classification of new, unknown instances.

The chosen method is a scoring mechanism that emits a *signal* $G_q^t$ for each possible taxon $t$ with regard to a given query sequence $q$. This signal represents the likelihood *given the available information* that $q$ belongs to any $t$. The process culminates by selecting the taxon with the highest signal value as $q_y$.

#### Signal calculation
When classifying a query sequence $q$, the agent iterates through all taxa present in the reference databases and calculates the corresponding signal score $G_q^t$.

For any given taxon, the agent iterates through the sites that hold information (type 1, 2 and 3), and based on the values presented by $q$ in these sites, a signal value for $t$ is emitted.

In type 1 and 2 sites, $t$ may present distinctive values that differentiate its representatives from the restof the taxa. If $q$ shares one of these values, an arbitrarily high signal value can be emitted.

It may happen that $t$ does not present distinctive values within the scope of the alignment. In this case a *composite signal* may be present if there exists a set of sites (type 2 and 3) that present a combination of values that is specific to $t$.

In order to be able to consider each site sepparately, a $5 \times 5 \times n$ matrix is defined, where each of the $5 \times 5$ submatrices corresponds to a given site of the alignment. If a site contains a *distinctive* value, the cell corresponding to said value in the site submatrix would contain the appropriate signal value.

Take the query sequence $q$ and $r^t \ in R^t$. The signal value for taxon $t$ would be calculated as:
$$G_q^t = \sum_{i \in n} M[q_i, r_i^t, i]$$

#### Matrix construction
The signal matrix $M_{sig}$ is first defined as:
$$M_{sig} = 0_{5,5,n}$$

The signal values that will populate $M_{sig}$ are retrieved as follows:

##### Distinctive values
These values can be found in *type 1* and *type 2* sites. Let $S_{1,2}^t$ be the set of type 1 and type 2 sites for $t$. For each $s \in S_{1,2}^t$, let $V^s$ be the set of values that are distinctive of $t$ at site $s$.
$$M_{sig}[v,v,s] = 100\ \forall\ v \in V^s\ \forall\ s \in S_{1,2}^t$$

##### Non-distinctive values
For the non distinctive values that can be found in *type 2* and *type 3* sites it is necesary to determine if a *composite signal* can be found. Take $S_{2,3}^t$ as the set of *type 2* and *type 3* sites for $t$, each site $s \in S_{2,3}^t$ presents a value $s_{\tau}$ so that $R_{\tau}$ is the subset of reference sequences for which $s = s_{\tau}$, and it includes sequences of $R_t$ and $R_{!t}$:
$$R_{\tau} \cap R_t \cap R_{!t} \ne \emptyset$$

While $R_{\tau}$ may include all the representatives of $t$, it must always exclude a fraction of $R_{!t}$
$$R_{!t} \not\subset R_{\tau}$$

The intersection of $R_t$ and $R_{!t}$ only exists for $\tau$, none other value of $s$ is shared between $R_t$ and $R_{!t}$
$$R_t \cap R_{!t} = \emptyset\ \forall\ v \in s; v \ne \tau$$

The *composite signal* of $t$, is determined as the set $\nu_t = \{\nu_1, \nu_2, ...\}$ where each site $\nu_i \in \nu_t$ contains a *non distinctive value* of $t$ and the intersection of all subsets of the alignment that cotain one of these values in one of the sites includes only sequences of $t$:
$$\bigcap_{i \in \nu_t} \nu_i = v_i \subseteq R_t$$

For each site $\nu_i \in \nu_t$ the corresponding position in $M_{sig}$ is updated with a value of $\frac{100}{|\nu_t|}$

The algorithm used to define the $\nu_t$ set is as follows:

    Of all potential sites, select the one with the least shared values outside of t
    Mark all sequences without observations in the selected site as solved (they are discriminated by the selected site)
    If there are no unsolved sequences left, terminate search, mark signal as solved
    Recount shared observations, excluding solved sequences
    If there are no sites to search, terminate search and mark signal as unsolved, return list of unsolvable sequences
    Return to beginning, if there is a tie for the site with the least observations, select the one with the least amount of solved taxa to minimize shared signal

# Program modules
Below is a brief description of the modules to be found in the graboid program

## Module *data_holder*
This module contains the DataHolder class, which is used to load reference and query datasets, select specific regions of the alignment, filter by coverage and collapse effective sequences.

### Class *DataHolder*
This class fulfills the tasks:
* Load reference data
* Load query data
* Select alignment region
* Filter selected region by coverage
* Collapse effective sequences

#### Usage
##### Load reference dataset
>load_reference(ref_dir, min_coverage=.8, required_rank='family')

Load the reference dataset. This step must be performed before loading the query dataset.

##### Load query
>load_query(qry_file, qry_dir='.', evalue=0.0005, dropoff=0.05, min_height=0.1, min_width=2, threads=1, qry_name='QUERY', min_coverage=.95)

Load the query sequences and build the query alignment.

Alternatively, use the method *load_query_quick* to load a pre-existing query alignment, skipping the alignment stage.
>load_query_quick(qry_map_file, qry_acc_file, min_coverage=.95)

##### Reset filters (optional)
>set_required_rank(required_rank='family')

Change the lowest known rank required for reference sequences.

>set_filters(min_cov_R=None, min_cov_Q=None)

Change minimum coverage filters for the reference or query datasets.

##### Get overlapping region between reference and query datasets
>get_overlap(min_cov_R=None, min_cov_Q=None, start=0, end=-1, collapse=True, max_unk_thresh=.2)

Select regions with enough coverage on both datasets. Arguments *min_cov_R* and *min_cov_Q* allow to reset the minimum coverage thresholds before selecting overlaps. Arguments *start* and *end* allow selection of a bounded region of the overlap (if it exists).

##### Select a specific region of the reference alignment
>select_region(start=0, end=-1, max_unk_thresh=.2)

Arguments *start* and *end* are used to determine the selected region. Selected region is collapsed, argument *max_unk_thresh* is sets the maximum threshold of unknown values for reference sequences to be included in an effective sequence cluster. Finally, *collapse* and *max_unk_thresh* allow to incorporate the collapsing of both dataframes right after overlapping.

##### Collapse sequences
>collapse(max_unk_thresh=.2)

Collapse both reference and query datasets. **NOTE: ensure that both datasets are up to date**

##### Access attributes
After applying coverage filters, selecting alignment sub-regions and collapsing effective sequences, data may be accessed in the following attributes:
* R.collapsed : Collapsed reference alignment
* R.y_collapsed : Consensus taxa for the collapsed effective sequences
* R.lineage_collapsed : Extended lineage of the collapsed effective sequences
* Q.collapsed : Collapsed query alignment
* Q.branch map : Series identifying member sequences of each collapsed branch

### Class *Data*
This abstract class hosts alignment datasets. It is tasked with:
* Filtering alignment sites by a minimum coverage threshold
* Selecting a given region of the alignment
* Collapsing effective sequences in the selected region of the alignment

### Class *R(Data)*
This class is a child of *Data* and hosts the reference dataset. It handles:
* Loading the reference alignment matrix and associated taxonomic data files
* Applying a second filter to select only reference sequences of known classification at a given rank
* Building consensus taxonomies after collapsing effective sequences

### Class *Q(Data)*
This class is a child of *Data* and hosts the query dataset. It handles:
* Loading the query sequence file (in Fasta format) and directing construction of its alignment matrix
* Loading a pre-existing query alignment matrix
* Building a map of collapsed sequences, recording the cluster to which each sequence is assigned

##### Loading reference datasets
A *graboid* reference dataset is comprised mainly of an alignment matrix $R$ and a taxonomic array $R_y$. Both $R$ and $R_y$ are *numerical* arrays containing data for *r* reference sequences. $R_y$ contains the numeric ID for each sequence for each taxonomic rank included in the database, its shape is $r \times \#Rk$.

The reference dataset requires ancillary data: An **accession list** detailing the repository ID codes for each sequence, **lineage table** detailing the lineage of each taxon, and a **names table** containing the human readable name assigned to each taxonomic ID.

Additionally, the reference dataset must also include a **guide sequence** used to build the alignment. This sequence will also be used to map the query dataset.

##### Loading query datasets
A query dataset consists of a fasta file containing DNA sequences to be classified. When a query dataset is loaded into graboid, the query sequences must be mapped against the reference **guide sequence**. This alignment is transformed into the numeric matrix $Q$. The fasta sequence identifiers are extracted into a list that will be used to identify individual sequences.

When building $Q$, the *DataHolder* keeps track of the boundaries of the query alignemnt. This will be used to match the query and reference datasets.

##### Filter data
Columns in the alignments can be filtered by *coverage* (those columns with a count of mising data over a given threshold). Reference sequences can be filtered by *taxonomic identification* (only keep sequences that are known at certain taxonomic rank).

##### Selecting columns from the reference datasets
A range of columns can be selected from the reference dataset. This can be done manually for testing purposes, but generally it will be performed automatically when matchin the query and reference datasets.

##### Matching reference & query datasets
In order to be able to classify the query sequences, both reference an query datasets must contain homologous regions. Using the stored boundaries of the query map, the appropriate subregion of the reference alignment can be selected.

### Methods
#### load_ref
##### Arguments
* ref_dir: *str*. Path to the directory containing the reference dataset files
##### Returns
* R: *numpy array*. Reference alignment stored as a 2d numeric matrix
* R_y: *numpy array*. Lowest known taxon ID for each sequence contained in $R$
* R_accs: *list*. List of accession codes of the sequences contained in $R$
* lineage_tab: *pandas DataFrame*. Table containing the lineages of every taxon included in the reference dataset
* names_tab: *pandas DataFrane*. Table mapping each taxon ID with its corresponding name
* R_coverage: *numpy array*. Array counting *known* values per site in the alignment
* R_coverage_norm: *numpy array*. Normalized coverage array.

This method looks for the required files in the provided *ref_dir* directory and attempts to load the refrence dataset map along with the ancillary files. Raises an exception if any file is missing or a conflict is detected (multiple files sharing the same suffix are detected).

Calls functions: *load_ref_files*, *load_map*

#### load_query
##### Arguments
* query_file: *str*. path to the fasta file containing the query DNA sequences
* blast_db: *str*. path to the blast database used to build the reference dataset
* query_dir: *str*, default: '.'. Path to the directory where the generated alignment will be stored
* evalue: *float*, default: 0.0005. E-value threshold for the query sequence alignment
* dropoff: *float*, default: 0.05. Relative coverage drop threshold to define coverage mesas
* min_height: *float*, default: 0.1. Minimum relative coverage required to define a coverage mesa
* min_width: *int*, default: 2. Minimum sites required to define a coverage mesa
* threads: *int*, default: 1. Threads to be used in the blast alignment
* qry_name: *str*, default: 'QUERY'. Suffix of the generated alignemt files
##### Returns
* Q: *numpy array*. Query alignment stored as a 2d numeric matrix
* Q_accs: *list*. List of accession codes of the sequences contained in $Q$
* Q_bounds: *list*. Bound coordinates of the scope of the $Q$ array
* Q_coverage: *numpy array*. Array counting *known* values per site in the alignment
* Q_coverage_norm: *numpy array*. Normalized coverage array.

This method performs the alignment of the query sequences and loads the generated alignment.

Calls functions: *map_query*, *load_map*


#### filter_data
##### Arguments
* min_cov: *float*, default: 0.95. Minimum coverage threshold
* rank: *str*, default: 'family'. Lowest known rank


#### select_sites
##### Arguments
##### Returns

## Concept learning agent

In this section I will design the agent that handles the concept learning and classification

### Class ConceptLearner

This is a director class. It's tasks include reference data loading and preprocessing; organizing Rank and Concept instances; and propagating learning and classification commands.

#### Attributes
* matrix: One-hot encoded alignment matrix
* lineage_tab: Data frame containing lineage information for each reference instance
* lineage_flat: Flatteed lineage dataframe used to look up instances belonging to a given taxon
* ranks: dictionary containing Rank instances

#### Methods
##### load_data
Arguments:

* matrix: Reference sequence alignment
* lineage_tab: Reference lineage data frame

This method loads and preprocess reference data, and prepares the *ranks* dictionary.

##### learn
Arguments:

* threads: Number of processors to employ when learning concepts

Propagates the learning command to each Rank instance and the Concepts they contain.

##### classify
Arguments:

* Q: Query sequence alignment
* threads: Number of processors to employ when classifying instances

### Associated functions
#### one_hot_encode
Arguments:

* matrix: Sequence alignment matrix

Returns:

* encoded: Encoded alignment matrix

This function encodes the given 2D alignment matrix of shape $\#sequences \times \#sites$ and values 1, 2, 3, 4 (A, C, G, T) into a 3D boolean alignment of shape $\#sequences \times \#sites \times 4$. Each of the 4 matrices at the third axis corresponds to one of the DNA bases and indicates whether a given sequence presents said base at a given site.

#### flatten_lineage
Arguments:

* R_lineage: Reference lineage dataframe

Returns:

* lineage_flat: Data frame with columns *idx*, *TaxId*

Processes the lineage table, generates a one column data frame with TaxIds as repeating indexes. Each TaxId points to all instances belonging to said taxon.

#### count_value_differences
Arguments:

* R_encoded:
* R_lineage:

Returns:

* count_diff
* full_count
* taxa

#### build_type_tab
Arguments:

* R_encoded
* R_lineage

Returns:

* type_tab
* type_tab_summ

### Class Rank
This class holds the Concept instances for all the taxa belonging to a given rank. The Rank's signal matrix is built from the learned Concepts.

#### Attibutes
* name: Name of the rank
* taxa: Dictionary containing Concept instances for every taxon found in the rank
* scores_tab: Data frame containing calculated signal scores for each concept present in the rank
* shared_tab: Data frame counting the number of sites with shared values (present in other taxa) for each concept
* signal_matrix: 3D matrix of shape $\# reference\ sequences \times 5 \times 5$, containing the calculated scores for each value pair for each site in the reference alignment
* summary
* confusion

#### Methods
##### learn
Arguments:

* matrix: Encoded reference alignment matrix
* lineage_tab: Reference lineage data frame
* lineage_flat: Flattened reference lineage
* threads: number of processes to employ when learning processes

Propagates the learn command to every Concept instance contained in *taxa*. Additionally, counts shared values among concepts and calculates signal scores for each concept. Finally, constructs summary and confusion matrix.

##### classify
Arguments:

* Q: Query sequences alignment
* threads: Number of threads to employ when classifying instances

Returns:

* signal_tab

Calculates the signal score for each concept for each query sequence. Scores are normalized with the equation
$$score / concept.info\_score$$
Constructs a signal table with the Raw and Normalized score for each concept for each query sequence. A classification is assigned based on the concept with the *highest normalized score*.

### Associated functions
#### build_concept
Arguments:

* taxon: TaxId of the taxon concept to be learned
* rank_name: Name of the taxon's rank
* tax_idxs: indexes of reference instances belonging to the taxon
* lineage_tab: Reference lineage data frame
* matrix: Encoded reference alignment matrix

Returns:

* concept_tax: Concept instance for the taxon

Generates a Concept instance and learns the characteristic rules for the given taxon

#### get_shared_info
Arguments:

* taxa: Dictionary containing the taxon Concept instances present in a given rank

Returns:

* shared_tab: Data frame with columns *Max_shared*, *Rem_shared*, *Total_shared*, *Non_shared* indicating the amount of shared sites in each concept's composite signal.
* non_shared: Dictionary containing non-shared sites for each taxon.

Collect information about the sites shared among a given rank's concepts' composite signals this information will be used in the calculation for the signal scores.

#### get_signal_scores
Arguments:

* shared_tab: Shared sites data returned by *get_shared_info*

Returns:

* scores_tab: Data frame with columns *NS_score*, *Info_score* containing calculated scores for shared and distinctive sites in each concept taxon.

Calculates the signal scores for the distinctive and composite signal sites in each concept.

#### build_signal_mat
Arguments:

* matrix: Encoded reference sequence alignment
* taxa: Dictionary containing the taxon Concept instances present in a given rank
* scores_tab: Data frame containing calculated scores for each concept, returned by *get_signal_scores*

Return:

* signal_matrix: 3D array of shape $\# sites \times 5 \times 5$ containing the signal scores for each value pair for each site

Constructs the signal matrix to be utilized for concept identification from query sequences

#### get_concept_signal
Arguments:

* Q: Query sequence alignment
* M: Signal matrix
* sn_sites: Array containing the indexes of the single value sites in the current Concept
* sn_values: Array containing the single values present in the current Concept
* ml_sites: Array containing the indexes of the multiple value sites in the current Concept
* ml_values: 2D array containing all unique value combinations in the multiple value sites in the current Concept

Returns:

* signal_total: Array containing the calculated concept signal for each query sequence

Calculate the signal for each query sequence in Q for a given concept

### Class Concept
This class holds the data of a given *Taxon* (Concept), it handles identification of informative sites and composite signal calculation.

#### Attributes
##### name
TaxId of the concept taxon.
##### rank
Taxonomic rank of the concept taxon.
##### sequences
Array containing indexes of the taxon's representative sequences in the reference alignment.
##### out_sequences
Array containing the indexes of sequences not belonging to the concept taxon in the reference alignment.
##### solved
String indicating the grade at which the contept taxon can be identified.
| Status | Description |
| :-: | :- |
|No|No sequences from the concept taxon can be differentiated from the rest of the reference alignment|
|Partial|Some of the concept taxon sequences can be differentiated from some of the sequences from the reference alignment|
|Full|All the concept taxon sequences can be differentiated from the rest of the reference alignment|
##### n_rules
Number of rules defined for the concept taxon.
##### rule_indexes
Array used to combine the rulesets of multiple concept taxa within the same given rank.
##### rule_sites
Array containing the position of the alignment sites employed as the concept rules.
##### rule_values
Boolean array indicating the valid values for every defined rule. The array has shape $|rules| \times 5$.
##### ruleset_origin
Boolean array indicating the ruleset that originates each of the rules (these may be the *Single full*, *Single partial* or *Composite* rulesets). The array has shape $|rules| \times 3$.
##### confirmed
Boolean array indicating which of the sub rulesets can confirm (separate from the rest of the alignment) each of the concept taxon's representative sequences. The array has shape $|sequences| \times 3$.
##### out_compatible
Boolean array indicating which of the outsider sequences are compatible with the generated rulesets. The array has shape $|out_sequences| \times 3$.
##### types
Array containing the number of each site type found for the concept taxon.
##### confirmed_seqs
Array containing the indexes of the concept taxon sequences that are confirmed by any of the generated rulesets.
##### out_compatible_seqs
Array containing the indexes of the outsider sequences that are compatible with all the generated rulesets.

#### Methods
##### learn
Arguments:
* matrix: Encoded reference sequence alignment matrix
* concept_sequences: Array of indexes of the taxon's representative sequences in the reference alignment

Detect site types for the concept taxon and define the concept's rulesets.

##### set_rule_indexes
Arguments:
* offset: Current length of the rank's merged ruleset used to correctly position the concept's rules.

Generate an array indicating the position of the concept taxon's ruleset in the rank's merged rulesets.

### Associated functions
#### get_distinct_vals
Arguments:

* matrix: Encoded reference alignment matrix
* in_indexes: Array of indexes of representative sequences of the concept taxon

Returns:

* distinct_sites: Array of sites containing distinctive values for the concept taxon
* distinct_vals: List of arrays containing the distinctive values found in each site. Contains the same number of elements as distinct_sites, each site may contain multiple distinctive values
* solved: Array of indexes of solved (fully distinguished from the outsider sequences by at least one distinctive value) concept taxon sequences
* unsolved: Array of indexes of concept taxon sequences without distinctive values

Identify all sequences within the in_indexes-defined group that contain at least one distinctive site for the concept taxon

#### get_composite_signal
Arguments:
* matrix: Encoded reference alignment matrix
* in_indexes: Array of indexes of representative sequences of the concept taxon
* unsolved_indexes: Array of indexes of concept taxon sequences without distinctive values

Returns:
* signal_v: Array of site indexes included in the *vertical* signal
* values_v: Array of semi-distinctive values of each site in the *vertical* signal
* signal_h: Array of site indexes included in the *horizontal* signal
* values_h: Array of semi-distinctive values of each site in the *horizontal* signal
* intersection_v: Array of outsider sequences indexes not differentiated by the *vertical* signal
* intersection_h: Array of outsider sequences indexes not differentiated by the *horizontal* signal
* seqs_h: Array of concept sequences solved by the *horizontal* signal. This array may contain incomplete concept sequences that nonetheless have known values in every site of the *horizontal* signal

Identify taxon signals that allow to differentiate unsolved concept sequences from the greatest number of outsider sequences

Vertical signal: determined using only full sites (columns) of concept taxon alignment

Horizontal signal: calculated using only full sequences (rows) of the contept taxon alignment
    
#### get_unsolved_subtaxa
Arguments:
* seqs_h: Array of concept sequences solved by the *horizontal* signal
* seqs_v: Array of concept sequences solved by the *vertical* signal
* in_indexes: Array containing the indexes of the concept sequences
* rank: Taxonomic rank of the concept
* lineage_tab: Data frame containing the reference lineage

Returns:
* solved_subtaxa:  Data frame counting the unsolved sequences for each subtaxon of the concept. Also includes the total count of sequences for each subtaxon
        
Count the number of unsolved sequences for vertical and horizontal signals
    
#### process_lineage
Arguments:

* rank: Rank column to be selected
* lineage_tab: Reference lineage data frame

Returns:
* lineage_processed: Two column dataframe containing the rank column with updated missing values (values missing in the leading rank are left as 0)
* rank_tail: Rank immediately below the selected one

Extracts the columns corresponding to rank and the one immediately below from the lineage table. Renames column names as Rank and Sub_rank. Replaces missing values in rank column for the values at the leading rank (unless rank is the highest). If the selected rank is the lowest one, both columns correspond to rank.

#### get_confused_out_taxa
Arguments:

* intersect_h: Array of outsider sequences indexes not differentiated by the *horizontal* signal
* intersect_v: Array of outsider sequences indexes not differentiated by the *vertical* signal
* in_indexes: Array containing the indexes of the concept sequences. Values must be between 0 and n_sequences - 1
* rank: Taxonomic rank of the concept
* lineage_tab: Reference lineage data frame

Returns:
* confused_subtaxa: Data frame counting the number of non distinguishable sequences for each outsider subtaxa. Also includes the total count of sequences for each subtaxon

Count the number  number of non distinguishable outsider sequences for vertical and horizontal signals

#### compress
Arguments:

* matrix: Encoded reference alignment matrix
* sequences: Array of concept sequences
* \*sites: Arrays of selected sites in the concept

Returns:

* sites_single: Array of indexes of selected sites with a single known value among the concept sequences
* values_single: Array of values present in the single value sites of the concept taxon
* sites_multi: Array of indexes of selected sites with multiple known values among the concept sequences
* values_multi: 2D array containing every unique combination of values present among the multiple value sites in the concept taxon

Separate variable and invraiable sites, extract representatives of each combination of values present in the concept sequences